To open this notebook in Colab visit https://goo.gl/magenta/groovae-colab

<img src="https://magenta-staging.tensorflow.org/assets/groovae/score-groove.png" alt="GrooVAE Figure" >

## TODO:
<ol>
  <li>NN to Max </li>
  <li>Python Module Everything</li>
  <li>Osc Integration</li>
  <li>Preserve Microtiming</li>
  <li>Control temperature with OSC</li>
  <li>What are mu, sigma?</li>
  <li>tpq=220</li>
</ol>

In [1]:
import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np
np.set_printoptions(suppress=True)

import tensorflow_datasets as tfds

import note_seq
from note_seq.protobuf import music_pb2

from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

GROOVAE_2BAR_TAP_FIXED_VELOCITY="groovae_2bar_tap_fixed_velocity.tar"

VELOCITY=85

/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
# If a sequence has notes at time before 0.0, scootch them up to 0
def start_notes_at_0(s):
    for n in s.notes:
        if n.start_time < 0:
            n.end_time -= n.start_time
            n.start_time = 0
    return s

# Some midi files come by default from different instrument channels
# Quick and dirty way to set midi files to be recognized as drums
def set_to_drums(ns):
    for n in ns.notes:
        n.instrument=9
        n.is_drum = True
        
# quickly change the tempo of a midi sequence and adjust all notes
def change_tempo(note_sequence, new_tempo):
    new_sequence = copy.deepcopy(note_sequence)
    ratio = note_sequence.tempos[0].qpm / new_tempo
    for note in new_sequence.notes:
        note.start_time = note.start_time * ratio
        note.end_time = note.end_time * ratio
    new_sequence.tempos[0].qpm = new_tempo
    return new_sequence

# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
    s_=copy.deepcopy(s)
    return note_seq.sequences_lib.quantize_note_sequence(s_,steps_per_quarter)

# Destructively quantize a midi sequence
def flatten_quantization(s):
    beat_length = 60. / s.tempos[0].qpm
    step_length = beat_length / 4 #s.quantization_info.steps_per_quarter
    new_s = copy.deepcopy(s)
    for note in new_s.notes:
        note.start_time = step_length * note.quantized_start_step
        note.end_time = step_length * note.quantized_end_step
    return new_s    

def add_silent_note(note_sequence, num_bars):
    tempo = note_sequence.tempos[0].qpm
    length = 60/tempo * 4 * num_bars
    note_sequence.notes.add(
        instrument=9, pitch=42, velocity=0, start_time=length-0.02, 
        end_time=length-0.01, is_drum=True)

def is_4_4(s):
    ts = s.time_signatures[0]
    return (ts.numerator == 4 and ts.denominator ==4)

# quick method for turning a drumbeat into a tapped rhythm
def get_tapped_2bar(s, velocity=VELOCITY, ride=False):
    new_s = dc_tap.from_tensors(dc_tap.to_tensors(s).inputs)[0]
    new_s = change_tempo(new_s, s.tempos[0].qpm)
    if velocity != 0:
        for n in new_s.notes:
            n.velocity = velocity
    if ride:
        for n in new_s.notes:
            n.pitch = 42
    return new_s

def drumify(s, model, temperature=1.0): 
    encoding, mu, sigma = model.encode([s])
    decoded = model.decode(encoding, length=32, temperature=temperature)
    return decoded[0]    

In [3]:
# Load MIDI files from GMD with MIDI only (no audio) as a tf.data.Dataset
dataset_2bar = tfds.as_numpy(tfds.load(
                name="groove/2bar-midionly",
                split=tfds.Split.VALIDATION,
                try_gcs=True))

dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]
print(len(dev_sequences))

2022-03-16 13:24:12.473343: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2022-03-16 13:24:18.936081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 13:24:18.945759: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-16 13:24:18.945781: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Can

2039


# Tap2Drum: Generate a beat from any rhythm 

While the Groove model works by removing the micro-timing and velocity information and learning to predict them from just the drum pattern, we can also go in the opposite direction.  Here, we take a representation of a Groove as input (in the form of a rhythm that can have precise timing but where drum categories are ignored) - and then generate drum beats that match the groove implied by this rhythm.  We trained this model by collapsing all drum hits from each beat in the training data to a single "tapped" rhythm, and then learning to decode full beats from that rhythm.  This allows us to input any rhythm we like through the precise onset timings in a "tap" and let the model decode our rhythm into a beat. We can even simply record taps as audio, or extract them from a recording of another instrument, rather than needing a midi controller.

Here are a couple of examples using MIDI rhythms:

In [4]:
dc_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'].data_converter

groovae_2bar_tap = TrainedModel(config=configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'],
                                batch_size=1,
                                checkpoint_dir_or_path=GROOVAE_2BAR_TAP_FIXED_VELOCITY)

sequence_indices = [1111, 366]
for i in sequence_indices:
    s = start_notes_at_0(dev_sequences[i])
    s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), dev_sequences[i].tempos[0].qpm)
    h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Decoder Cells:
  units: [256, 256]



...

/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
Instructions for updating:
Do not call `graph_parents`.


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Restoring parameters from /tmp/tmpzwjjxrkm/groovae_2bar_tap_fixed_velocity/model.ckpt-3668


INFO:tensorflow:Restoring parameters from /tmp/tmpzwjjxrkm/groovae_2bar_tap_fixed_velocity/model.ckpt-3668


# IO Communication

quantized_start_step, quantized_end_step  
control_changes {
  control_number: 4
  control_value: 90
  is_drum: true
}

In [10]:
def max_list_to_midi_array(max_list, BPM, n_bars=2, n_steps_per_quarter_note=4):
    """max_list timing are in bars. Assumes 4/4 timing"""
    assert len(max_list)==3*(n_bars*4*n_steps_per_quarter_note), 'List length wrong!'
    beat_dur=60/BPM # in sec
    midi_array=[]
    for i in range((len(max_list)//3)):
        start_step=4*float(max_list[3*i]) # in beats
        end_step=4*float(max_list[3*i+1]) # in beats
        vel=float(max_list[3*i+2])
        start_time=start_step*beat_dur
        end_time=end_step*beat_dur
        midi_array.append([start_time,end_time,vel])
    return np.array(midi_array)

def make_tap_sequence(midi_array, tempo, tpq=480):
    note_sequence=music_pb2.NoteSequence()
    note_sequence.tempos.add(qpm=tempo)
    note_sequence.ticks_per_quarter=tpq
    note_sequence.time_signatures.add(numerator=4, denominator=4)
    note_sequence.key_signatures.add()
    for onset_time, offset_time, onset_velocity in midi_array:
        if onset_velocity: # Non-zero velocity notes only
            note_sequence.notes.add(instrument=9, # Drum MIDI Program number
                                    pitch=42, # Constant
                                    is_drum=True,
                                    velocity=VELOCITY,
                                    start_time=onset_time,
                                    end_time=offset_time)
    note_sequence.total_time=2*4*(60/BPM) # 2bars
    return note_sequence 

def quantize_to_beat_divisions(beat, division=32):
    """Quantize a floating point beat? to a 1/division'th beat"""
    if division!=1: 
        return (beat//(1/division))*(1/division)
    else: # do not quantize
        return beat

def NN_output_to_Max(h, BPM, pre_quantization=False, beat_quantization_division=1):
    """Return in [beats]"""
    _h=copy.deepcopy(h)
    beat_dur=60/BPM
    if pre_quantization:
        _h=quantize(_h)
    midi_array=[]
    for note in _h.notes:
        start=quantize_to_beat_divisions(note.start_time/beat_dur, beat_quantization_division)
        dur=quantize_to_beat_divisions((note.end_time-note.start_time)/beat_dur, beat_quantization_division)
        midi_array.append([start, dur, note.velocity, note.pitch])
    midi_array=np.array(midi_array)
    return midi_array     

In [11]:
max_lst="0. 0.03 120. 0.0625 0.0925 120. 0.125 0.155 120. 0.1875 0.2175 120. 0.25 0.28 120. 0.3125 0.3425 120. 0.375 0.405 0. 0.4375 0.4675 0. 0.5 0.53 0. 0.5625 0.5925 0. 0.625 0.655 0. 0.6875 0.7175 120. 0.75 0.78 0. 0.8125 0.8425 0. 0.875 0.905 120. 0.9375 0.9675 0. 1. 1.03 120. 1.0625 1.0925 120. 1.125 1.155 0. 1.1875 1.2175 0. 1.25 1.28 0. 1.3125 1.3425 0. 1.375 1.405 0. 1.4375 1.4675 0. 1.5 1.53 0. 1.5625 1.5925 0. 1.625 1.655 0. 1.6875 1.7175 0. 1.75 1.78 120. 1.8125 1.8425 120. 1.875 1.905 120. 1.9375 1.9675 120.".split(' ')

BPM=120
# Convert max list to a readable format
midi_array=max_list_to_midi_array(max_lst, BPM)
len(midi_array)
# Convert it into the pre-NN input format
note_sequence=make_tap_sequence(midi_array, BPM)
note_sequence=quantize(note_sequence)
set_to_drums(note_sequence)
# Convert to NN input format
s=copy.deepcopy(note_sequence)
s=start_notes_at_0(s)
s=change_tempo(get_tapped_2bar(s, velocity=VELOCITY, ride=True), BPM)
assert BPM==s.tempos[0].qpm, 'Tempo conversion failed at tapped bar creation'
# Get NN prediction
h=change_tempo(drumify(s, groovae_2bar_tap), BPM)
assert BPM==h.tempos[0].qpm, 'Tempo conversion failed at NN creation'
# Convert to Max array
MAX_array=NN_output_to_Max(h, BPM, beat_quantization_division=32)

In [12]:
MAX_array

array([[ 0.     ,  0.25   , 17.     , 38.     ],
       [ 0.21875,  0.25   , 57.     , 38.     ],
       [ 0.21875,  0.25   , 28.     , 50.     ],
       [ 0.46875,  0.25   , 57.     , 38.     ],
       [ 0.75   ,  0.25   , 18.     , 36.     ],
       [ 0.71875,  0.25   , 45.     , 38.     ],
       [ 0.96875,  0.25   , 51.     , 38.     ],
       [ 1.25   ,  0.25   , 12.     , 36.     ],
       [ 1.96875,  0.25   , 26.     , 38.     ],
       [ 2.75   ,  0.25   ,  6.     , 48.     ],
       [ 2.96875,  0.25   , 30.     , 38.     ],
       [ 3.25   ,  0.25   , 45.     , 38.     ],
       [ 3.5    ,  0.25   , 36.     , 38.     ],
       [ 3.96875,  0.25   , 47.     , 36.     ],
       [ 6.     ,  0.25   , 45.     , 38.     ],
       [ 6.25   ,  0.25   , 21.     , 38.     ],
       [ 6.5    ,  0.25   , 11.     , 38.     ],
       [ 6.71875,  0.25   ,  4.     , 36.     ],
       [ 6.71875,  0.25   , 25.     , 38.     ],
       [ 6.96875,  0.25   , 44.     , 38.     ],
       [ 7.25   ,  0